In [4]:
%%sh
cat > pi-mpi.c << EOF
#include <stdio.h>
#include <mpi.h>

int main(int argc, char *argv[]) {
    int rank, size;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    int start = rank * 10 + 1;
    int end = (rank + 1) * 10 + 1;
    int sum = 0;
    for (int i = start; i < end; i++) {
        sum += i;
    }
    double average = (double) sum / (end - start);
    printf("Proces %d: Suma: %d Srednia: %.2f\n", rank, sum, average);

    int total_sum;
    if (rank == size - 1) {
        total_sum = sum;
        for (int i = 0; i < size - 1; i++) {
            int part_sum;
            MPI_Recv(&part_sum, 1, MPI_INT, i, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
            total_sum += part_sum;
        }
        printf("Proces %d: Suma calkowita: %d Srednia calkowita: %.2f\n", rank, total_sum, (double) total_sum / 100);
    } else {
        MPI_Send(&sum, 1, MPI_INT, size - 1, 0, MPI_COMM_WORLD);
    }

    MPI_Finalize();
    return 0;
}
EOF

mpicc pi-mpi.c && mpirun -n 1 --allow-run-as-root a.out 

Proces 0: Suma: 55 Srednia: 5.50
Proces 0: Suma calkowita: 55 Srednia calkowita: 0.55
